In [1]:
%store -r train_mod 
%store -r test_mod

In [2]:
train_mod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47661 entries, 0 to 88660
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              47661 non-null  object 
 1   brand                 47661 non-null  object 
 2   color                 47661 non-null  object 
 3   fuelType              47661 non-null  object 
 4   modelDate             47661 non-null  Int64  
 5   name                  47661 non-null  object 
 6   numberOfDoors         47661 non-null  Int64  
 7   productionDate        47661 non-null  int64  
 8   vehicleConfiguration  47661 non-null  object 
 9   vehicleTransmission   47661 non-null  object 
 10  engineDisplacement    30568 non-null  float64
 11  enginePower           47661 non-null  Int64  
 12  description           46081 non-null  object 
 13  mileage               47661 non-null  int64  
 14  Комплектация          47661 non-null  object 
 15  Привод             